# Import

In [21]:
import csv
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

## Add configuration file

In [22]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [23]:
from ALL import config 
from util import *

## Set condition

In [24]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [25]:
data_type="AgNews"

# Read data

In [26]:
df = pd.read_csv(
    f"../../Preprocessing/data/{data_type}/master.csv", index_col=0
)

In [27]:
# limitation for test
df = df.sample(1000)

In [28]:
with open(f"../../Preprocessing/data/{data_type}/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [29]:
max_model_num = config["vectorize"]["sentenceBERT"]["max_model_num"]

# Embedding

In [30]:
def get_sentenceBERT(texts,seed, path):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    model.save(path)
    vectors = model.encode(texts)
    return vectors

In [31]:
vectors_path = f"../data/{data_type}/sentenceBERT/vector/raw"
models_path = f"../data/{data_type}/sentenceBERT/model"

for model_num in range(max_model_num):
    vectors = get_sentenceBERT(
        df.text.tolist(),
        seed=model_num,
        path=make_filepath(f"{models_path}/{model_num}"),
    )

    np.save(
        make_filepath(f"{vectors_path}/{model_num}.npy"),
        np.stack(vectors),
    )

In [33]:
# import shutil
# shutil.rmtree('data/AgNews/sentenceBERT/')